In [ ]:
pd.__version__

In [ ]:
#!pip install --upgrade pandas

#### Logistic Model 7 SVM Model development for Statistical Analysis

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import warnings
from collections import Counter

from itertools import chain
from time import time

#Feature Selection 
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV , SelectFromModel
#classifiers
from xgboost import XGBClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVC
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report , confusion_matrix , accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder , OneHotEncoder

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import RandomizedSearchCV , cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.utils import resample#, suffle
import  matplotlib.pyplot as plt 
#from sklearn.utils import suffle

In [ ]:
#df1 = pd.read_csv("../case_snp.csv",dtype=object, low_memory=False)
df1 = pd.read_csv("../case_snp_10.csv",dtype = object,low_memory=False) #This is New Data 

In [ ]:
#df = pd.read_csv("../control_snp.csv",dtype=object,low_memory=False)
df = pd.read_csv("../con_10.csv",dtype=object,low_memory=False)

In [ ]:
df1.shape

In [ ]:
df.shape

In [ ]:
df1.head(2)

In [ ]:
df.CHROM.unique()

In [ ]:
df1.shape

Since there is very Big data so We Select Out only Highly influencing and different dataas to each Other

In [ ]:
plt.figure(figsize=(8,16))

In [ ]:
plt.figure(figsize=(16,8))
sns.countplot(x = "#CHROM",
             data = df,
             order = df["#CHROM"].value_counts().index)

In [ ]:
plt.figure(figsize=(16,8))
sns.countplot(x = "#CHROM",
             data = df1,
             order = df1["#CHROM"].value_counts().index)

In [ ]:
case_con = df[(df.iloc[:,[4,5,6,7,8,9,10,11,12,13]].isnull().sum(axis =1)<2)
    &(df.iloc[:,[14,15,16,17,18,19,20,21,22,23]].isnull().sum(axis =1)>9)] #We Can accept up to  80 %

#con_case = df[(df.iloc[:,[4,5,6,7,8,9,10,11,12,13]].isnull().sum(axis =1)>9) \
#    &(df.iloc[:,[14,15,16,17,18,19,20,21,22,23]].isnull().sum(axis =1)<2)]

In [ ]:
#Here NaN is Assumed as same Values as Ref means No SNP at that Position
# Currently SNP in Control is not Useful for our purpose of Analysis
#But we may think these types of SNP has Positive impact

In [ ]:
case_con

In [ ]:
case_con.shape
#This 20 Rows are Highly sensitive because these are recurring only in Case more than 90 %

In [ ]:
#Since 

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.title("Heat Map ")
plt.xlabel("Case(c) & Control(cn)")
plt.ylabel("Position of Sequence (SNP)")

sns.heatmap(case_con.iloc[:,[4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]].isnull(),cmap = "viridis",\
            cbar = False )

In [ ]:
#In this Figure we Can see yello Portions is Not Snp and brown in SNP
#We are Planning to Apply Logistic Regression Method if Our Data set increase more than 30/40 sets
# Till now we cant make decision 

In [ ]:
rep_chrom = case_con.groupby(["CHROM"]).size()
rep_chrom.plot()

In [ ]:
sns.countplot(case_con["CHROM"])

In [ ]:
#df_gr = df.groupby(["CHROM"]).size()

In [ ]:
#df_gr#We can see the Number of Chromosome repeating in df data set

In [ ]:
df.head(2)

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.title("Heat Map ")
plt.xlabel("Case(c) & Control(cn)")
plt.ylabel("Position of Sequence (SNP)")

sns.heatmap(con_case.iloc[:,[4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]].isnull(),cmap = "viridis",\
            cbar = False )

## Method 2, 

In [ ]:
test = df.head()

In [ ]:
#test["Case"] = test.iloc[:,4:14].apply(lambda x: ",".join(x.astype(str)), axis=1)
#df["Case"] = df.iloc[:,4:14].apply(lambda x: ",".join(x.astype(str)), axis=1)
#The Output is Same but Processing time is So slow
#Dont Recommend this method

In [ ]:
#df= df.assign(Case = df["1c"].astype(str)+',' +df["2c"].astype(str)+','+df["3c"].astype(str)+','+df["4c"].astype(str)\
#               + ', '+df["5c"].astype(str)+','+df["6c"].astype(str)+',' +df["7c"].astype(str)+','+df["8c"].astype(str)+','\
#              +df["9c"].astype(str)+','+df["10c"].astype(str))

In [ ]:
#df.assign() method is fast

In [ ]:
df= df.assign(Control = df["1cn"].astype(str)+',' +df["2cn"].astype(str)+','+df["3cn"].astype(str)+','+df["4cn"].astype(str)\
               + ', '+df["5cn"].astype(str)+','+df["6cn"].astype(str)+',' +df["7cn"].astype(str)+','+df["8cn"].astype(str)+','\
              +df["9cn"].astype(str)+','+df["10cn"].astype(str))

In [ ]:
#df["Control"] = df.iloc[:,4:14].apply(lambda x: ",".join(x.astype(str)), axis=1)
#This Method seems to time Consuming
#test = test.assign(Control = test["1cn"].astype(str) + ', ' + \
#  test["2cn"].astype(str) + ', ' +test["3cn"].astype(str) + ', ' \
#  +test["4cn"].astype(str) + ', '+test["5cn"].astype(str))

#df= df.assign(Control = df["1cn"].astype(str)+',' +df["2cn"].astype(str)+','+df["3cn"].astype(str)+','+df["4cn"].astype(str)\
#               + ', '+df["5cn"].astype(str)+','+df["6cn"].astype(str)+',' +df["7cn"].astype(str)+','+df["8cn"].astype(str)+','\
#              +df["9cn"].astype(str)+','+df["10cn"].astype(str))

In [ ]:
df=df.iloc[:,[0,1,2,3,-2,-1]]
df.head(2)

In [ ]:
df.to_csv("../case_control_list.csv",index=False)
#test = df.head(100)

In [ ]:
#test = test.iloc[:,[0,1,2,3,14,15,16,17,18,19,20,21,22,23,24]]
#test = test.iloc[:,[0,1,2,3,-2,-1]]
#test.head(2)

In [ ]:
df.head()

In [ ]:
#df.drop_duplicates(keep="first")

In [ ]:
from collections import Counter
df["case_max"] = df["Case"].apply(lambda x :(Counter((list(x.split(","))))).most_common(1))
df["cont_max"] = df["Control"].apply(lambda x:(Counter(x.split(","))).most_common(1))
df.head()

In [ ]:
#the value inside case_max and cont_max is 
#Create a new Col and Drop same Col
df["Case_"]= df["case_max"].apply(lambda x : x[0])
df =df.drop("case_max",axis=1)

#Create a new Col and drop same Col.
df["Cont_"]= df["cont_max"].apply(lambda x : x[0])
df = df.drop("cont_max",axis =1)
df.head()

In [ ]:
df.shape

In [ ]:
test = df.head(100)
test= test.assign(Control_ = test["Case_"].astype(str)+',' +test["Cont_"].astype(str))
test.head(2)

In [ ]:
#df.to_csv("../case_control_list_max.csv",index = False)

In [ ]:
df.shape
#Now We Are Going to Filter remove datas based on 1 condition
#if case_max ==cont_max 
#if case and control F SNP is same and count value is Greater than 5

In [ ]:
#test = test.iloc[:,[0,1,2,3,4,5,-1]]
test["Result"] = test["Control_"].apply(lambda x : True if((x[0][0] ==x[1][0])) else False)# & (x[0][1] ==x[1][1])) else False)
test.head(2)

In [ ]:
x =test.Control_[0]

In [ ]:
x[1][0]

In [ ]:
#from collections import Counter
#test["case_max"] = test["Case"].apply(lambda x :(Counter((list(x.split(","))))).most_common(1))
#test["cont_max"] = test["Control"].apply(lambda x:(Counter(x.split(","))).most_common(1))
#test.head()

In [ ]:
#df = df.sort_values(by=["POS","ID"])

In [ ]:
#df = df.reset_index(drop=True)

In [ ]:
#df.head()

In [ ]:
#df.to_csv("../snp_analysis/merged_files/case_control_merged_index_reset.csv",index=False,encoding="utf-8")

In [ ]:
#df = pd.read_csv("../snp_analysis/merged_files/case_control_merged_index_reset.csv")

In [ ]:
#df.shape

In [ ]:
#df.head